## Exercise on Micrograd and Pytorch

In [1]:
from math import sin, cos, sqrt, log, exp

### section 1: derivatives

In [2]:
# here is a mathematical expression that takes 3 inputs and produces one output
def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


The math expression is 

$$
f(a, b, c) = -a^3 + \sin(3b) -\frac{1}{c} + \sqrt{b^5} - \sqrt{a}.
$$
Then, we have that
$$
\nabla f(a, b, c) = \left[\frac{\partial f}{\partial a}, \frac{\partial f}{\partial b}, \frac{\partial f}{\partial c}\right](a, b, c),
$$
with
- $\frac{\partial f}{\partial a}(a, b, c) = - \left(3a^2 + \frac{1}{2\sqrt{a}}\right)$;
- $\frac{\partial f}{\partial b}(a, b, c) = 3\cos(3b) + \frac{5}{2}\sqrt{b^3}$;
- $\frac{\partial f}{\partial c}(a, b, c) = \frac{1}{c^2}$;

In [3]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula

def gradf(a, b, c):
    daf = -(3 * (a**2) + 0.5 / sqrt(a))
    dbf = 3 * cos(3 * b) + 0.5 * 5 * sqrt(b**3)
    dcf = c**(-2)
    return [daf, dbf, dcf]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


Now, we estimate the derivatives with a right finite difference; given $0 < h \ll 1$
- $\frac{\partial f}{\partial a}(a, b, c) \approx \frac{f(a + h, b, c) - f(a, b, c)}{h}$;
- $\frac{\partial f}{\partial b}(a, b, c) \approx \frac{f(a, b + h, c) - f(a, b, c)}{h}$;
- $\frac{\partial f}{\partial c}(a, b, c) \approx \frac{f(a, b, c + h) - f(a, b, c)}{h}$;

In [4]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
def right_numerical_gradf(a, b, c, h = 1e-6):
  daf = lambda a, b, c: (f(a + h, b, c) - f(a, b, c))/h
  dbf = lambda a, b, c: (f(a, b + h, c) - f(a, b, c))/h
  dcf = lambda a, b, c: (f(a, b, c + h) - f(a, b, c))/h
  return [daf(a, b, c), dbf(a, b, c), dcf(a, b, c)]
  
numerical_grad = right_numerical_gradf(2, 3, 4)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}, absolute difference is {abs(ans[dim] - numerical_grad[dim]):.3e}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995, absolute difference is 5.958e-06
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482, absolute difference is 1.396e-06
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534, absolute difference is 1.526e-08


In [5]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
def numerical_gradf(a, b, c, h = 1e-3):
  daf = lambda a, b, c: (f(a + h, b, c) - f(a - h, b, c))/(2 * h)
  dbf = lambda a, b, c: (f(a, b + h, c) - f(a, b - h, c))/(2 * h)
  dcf = lambda a, b, c: (f(a, b, c + h) - f(a, b, c - h))/(2 * h)
  return [daf(a, b, c), dbf(a, b, c), dcf(a, b, c)]

numerical_grad2 = numerical_gradf(2, 3, 4)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}, absolute difference is {abs(ans[dim] - numerical_grad[dim]):.3e}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554401639766, absolute difference is 5.958e-06
OK for dim 1: expected 10.25699027111255, yours returns 10.256994551617105, absolute difference is 1.396e-06
OK for dim 2: expected 0.0625, yours returns 0.06250000390650712, absolute difference is 1.526e-08


### Section 2: support for softmax

In [6]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    
    out._backward = _backward
    return out
  
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
      
    out._backward = _backward
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "Only int/float exponents are supported"
    out = Value(self.data**other, (self, ), '**')
    
    def _backward():
      self.grad += other * (self.data**(other - 1)) * out.grad
      
    out._backward = _backward
    return out
  
  def __truediv__(self, other):
    return self * (other**(-1))
    
  
  def exp(self):
    out = Value(exp(self.data), (self, ), "exp")
    
    def _backward():
      self.grad += exp(self.data) * out.grad
      
    out._backward = _backward
    return out
  
  def log(self):
    out = Value(log(self.data), (self,), 'log')
    
    def _backward():
      self.grad += (self.data**(-1)) * out.grad
      
    out._backward = _backward
    return out

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()
      
  def __radd__(self, other): # other + self
    return self + other
  
  def __neg__(self):
    return self * (-1)

In [7]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986
